# Product DB — Colab Queries & Peer Review

In [ ]:

import os, getpass, pandas as pd
from sqlalchemy import create_engine, text

STUDENT_DB_URL = os.environ.get("NEON_STUDENT_DB_URL") or getpass.getpass("Paste STUDENT DB URL (hidden): ")
engine = create_engine(STUDENT_DB_URL, pool_pre_ping=True, future=True)

def run_query(sql: str) -> pd.DataFrame:
    with engine.connect() as conn:
        df = pd.read_sql_query(text(sql), conn)
    return df

pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", 20)
print("Engine ready. ✅")


In [ ]:

# Sanity check
counts_sql = '''
SELECT 'product' AS table, COUNT(*) AS rows FROM product
UNION ALL SELECT 'pc', COUNT(*) FROM pc
UNION ALL SELECT 'laptop', COUNT(*) FROM laptop
UNION ALL SELECT 'printer', COUNT(*) FROM printer;
'''
run_query(counts_sql)


In [ ]:

# Query 1
q1 = '''
WITH catalog AS (
  SELECT p.maker, p.model, p.type,
         COALESCE(pc.price, l.price, pr.price) AS price
  FROM product p
  LEFT JOIN pc      pc  ON pc.model  = p.model
  LEFT JOIN laptop  l   ON l.model   = p.model
  LEFT JOIN printer pr  ON pr.model  = p.model
  WHERE COALESCE(pc.price, l.price, pr.price) IS NOT NULL
),
ranked AS (
  SELECT maker, model, type, price,
         DENSE_RANK() OVER (PARTITION BY type ORDER BY price DESC) AS rnk
  FROM catalog
)
SELECT type, maker, model, price
FROM ranked
WHERE rnk <= 3
ORDER BY type, price DESC, maker, model;
'''
run_query(q1)


In [ ]:

# Query 2
q2 = '''
SELECT maker
FROM product p
GROUP BY maker
HAVING
  SUM(CASE WHEN type = 'pc' THEN 1 ELSE 0 END) > 0
  AND SUM(CASE WHEN type = 'laptop' THEN 1 ELSE 0 END) > 0
  AND SUM(CASE WHEN type = 'printer' THEN 1 ELSE 0 END) > 0
ORDER BY maker;
'''
run_query(q2)


In [ ]:

# Query 3
q3 = '''
SELECT p.maker, l.model, l.speed, l.ram, l.hd, l.screen, l.price,
       ROUND( (l.price / NULLIF(l.speed * (l.ram/1024.0), 0)), 2) AS value_score
FROM product p
JOIN laptop l USING (model)
WHERE l.speed IS NOT NULL AND l.ram IS NOT NULL AND l.price IS NOT NULL
ORDER BY value_score ASC, l.price ASC
LIMIT 15;
'''
run_query(q3)


In [ ]:

# Classmate Query 1
sql_1 = """
-- Paste your classmate's SQL here
"""
try:
    display(run_query(sql_1).head(50))
except Exception as e:
    print("Error:", e)


In [ ]:

# Classmate Query 2
sql_2 = """
-- Paste your classmate's SQL here
"""
try:
    display(run_query(sql_2).head(50))
except Exception as e:
    print("Error:", e)


In [ ]:

# Classmate Query 3
sql_3 = """
-- Paste your classmate's SQL here
"""
try:
    display(run_query(sql_3).head(50))
except Exception as e:
    print("Error:", e)
